# Detección de Cambios con Sentinel-1 en Municipios de Casanare y Meta
## Parte 2: Preprocesamiento y Filtrado de Imágenes Sentinel-1

### Introducción

Este notebook implementa el preprocesamiento de imágenes Sentinel-1 SAR siguiendo las mejores prácticas para análisis de series temporales [1]. Sentinel-1 proporciona imágenes de radar en banda C con polarización dual (VV y VH), que son particularmente útiles para monitoreo agrícola debido a su sensibilidad a la estructura y contenido de humedad de la vegetación [2].

### Misión Sentinel-1

Sentinel-1 es una constelación de dos satélites (S1A y S1B) que operan en banda C (5.405 GHz) con las siguientes características [3]:

- **Resolución espacial**: 10 m (modo IW - Interferometric Wide swath)
- **Resolución temporal**: 6-12 días (dependiendo de la latitud)
- **Polarizaciones**: VV, VH (dual-pol)
- **Ancho de barrido**: 250 km

### Preprocesamiento en Google Earth Engine

Las imágenes Sentinel-1 disponibles en GEE ya han sido preprocesadas por el sistema SNAP [4], incluyendo:
- Corrección de órbita
- Calibración radiométrica
- Corrección del terreno (GRD - Ground Range Detected)

En este notebook se aplicarán pasos adicionales de procesamiento:
1. Filtrado espacial (reducción de speckle)
2. Filtrado temporal
3. Composiciones temporales
4. Normalización

---

### Referencias

[1] M. J. Canty, A. A. Nielsen, H. Skriver, and K. Conradsen, "Statistical analysis of changes in Sentinel-1 time series on the Google Earth Engine," *Remote Sens.*, vol. 12, no. 1, p. 46, Jan. 2020.

[2] V. Veloso et al., "Understanding the temporal behavior of crops using Sentinel-1 and Sentinel-2-like data for agricultural applications," *Remote Sens. Environ.*, vol. 199, pp. 415–426, Sep. 2017.

[3] ESA, "Sentinel-1 User Handbook," European Space Agency, Tech. Rep. ESA-EOEP-CSCOP-TN-13-0001, 2013.

[4] L. Filipponi, "Sentinel-1 GRD preprocessing workflow," in *Proc. 3rd Int. Electron. Conf. Remote Sens.*, 2019, pp. 1–11.

## 1. Carga de Librerías y Datos del Notebook Anterior

In [ ]:
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Inicializar Earth Engine
try:
    ee.Initialize()
    print("Earth Engine inicializado correctamente")
except:
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine autenticado e inicializado")

In [ ]:
# Cargar datos del notebook anterior
municipios_seleccionados = gpd.read_file("data/municipios_seleccionados.gpkg", layer="municipios")

with open('data/parametros.json', 'r') as f:
    parametros = json.load(f)

print(f"Municipios cargados: {len(municipios_seleccionados)}")
print(f"Período: {parametros['fecha_inicio']} a {parametros['fecha_fin']}")
print(f"Área total: {parametros['area_km2']:.2f} km²")

In [ ]:
# Recrear AOI de Earth Engine
def gdf_to_ee_geometry(gdf):
    """Convierte un GeoDataFrame a una geometría de Earth Engine"""
    geom_union = gdf.geometry.unary_union
    if geom_union.geom_type == 'Polygon':
        coords = [list(geom_union.exterior.coords)]
        return ee.Geometry.Polygon(coords)
    elif geom_union.geom_type == 'MultiPolygon':
        coords = [list(poly.exterior.coords) for poly in geom_union.geoms]
        return ee.Geometry.MultiPolygon(coords)

aoi = gdf_to_ee_geometry(municipios_seleccionados)
print("AOI recreada en Earth Engine")

## 2. Acceso a la Colección Sentinel-1

In [ ]:
# Acceder a la colección Sentinel-1 GRD (Ground Range Detected)
# GRD: productos calibrados radiométricamente y corregidos por terreno

sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(parametros['fecha_inicio'], parametros['fecha_fin']) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide swath mode

# Obtener información de la colección
count = sentinel1.size().getInfo()
print(f"Número total de imágenes Sentinel-1 disponibles: {count}")

# Verificar las propiedades de las primeras imágenes
if count > 0:
    first_image = ee.Image(sentinel1.first())
    props = first_image.getInfo()['properties']
    print(f"\nPropiedades de la primera imagen:")
    print(f"  Plataforma: {props.get('platform_number', 'N/A')}")
    print(f"  Modo de instrumento: {props.get('instrumentMode', 'N/A')}")
    print(f"  Órbita: {props.get('orbitProperties_pass', 'N/A')}")
    print(f"  Polarizaciones: {props.get('transmitterReceiverPolarisation', 'N/A')}")

## 3. Filtrado por Polarización y Órbita

Para análisis de cambios consistente, es importante usar imágenes de la misma geometría de adquisición [1].

In [ ]:
# Filtrar por polarización VV y VH (dual-pol)
# VV: más sensible a estructura del dosel
# VH: más sensible a biomasa y contenido de humedad

s1_vv_vh = sentinel1.filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
).filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')
)

# Separar por dirección de órbita (ascending/descending)
s1_asc = s1_vv_vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
s1_desc = s1_vv_vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

count_asc = s1_asc.size().getInfo()
count_desc = s1_desc.size().getInfo()

print(f"Imágenes con VV+VH:")
print(f"  Ascendentes: {count_asc}")
print(f"  Descendentes: {count_desc}")

# Seleccionar la órbita con más imágenes
if count_asc >= count_desc:
    s1_filtered = s1_asc
    orbit_type = 'ASCENDING'
    print(f"\nUsando órbita: ASCENDING ({count_asc} imágenes)")
else:
    s1_filtered = s1_desc
    orbit_type = 'DESCENDING'
    print(f"\nUsando órbita: DESCENDING ({count_desc} imágenes)")

## 4. Análisis Temporal de Disponibilidad de Datos

In [ ]:
# Obtener fechas de todas las imágenes
def get_image_dates(collection):
    """Extrae las fechas de una colección de imágenes"""
    def get_date(image):
        return ee.Feature(None, {'date': image.date().format('YYYY-MM-dd')})
    
    dates = collection.map(get_date).aggregate_array('date').getInfo()
    return sorted(dates)

fechas_disponibles = get_image_dates(s1_filtered)
print(f"Total de fechas: {len(fechas_disponibles)}")
print(f"Primera imagen: {fechas_disponibles[0]}")
print(f"Última imagen: {fechas_disponibles[-1]}")

# Calcular estadísticas temporales
fechas_dt = pd.to_datetime(fechas_disponibles)
diferencias = fechas_dt.to_series().diff().dt.days.dropna()

print(f"\nEstadísticas de revisita:")
print(f"  Media: {diferencias.mean():.1f} días")
print(f"  Mediana: {diferencias.median():.1f} días")
print(f"  Mínimo: {diferencias.min():.0f} días")
print(f"  Máximo: {diferencias.max():.0f} días")

In [ ]:
# Visualizar distribución temporal
plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.plot(fechas_dt, range(len(fechas_dt)), 'b-', linewidth=0.5)
plt.scatter(fechas_dt, range(len(fechas_dt)), s=10, c='red', alpha=0.5)
plt.xlabel('Fecha')
plt.ylabel('Número de imagen')
plt.title('Disponibilidad Temporal de Imágenes Sentinel-1')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.hist(diferencias, bins=20, edgecolor='black')
plt.xlabel('Días entre imágenes')
plt.ylabel('Frecuencia')
plt.title('Histograma de Período de Revisita')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data/temporal_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("Gráfico guardado en: data/temporal_distribution.png")

## 5. Conversión a dB y Selección de Bandas

Los valores de backscatter se trabajan típicamente en escala logarítmica (dB) para facilitar la interpretación [2].

In [ ]:
def to_dB(image):
    """Convierte valores de backscatter de lineal a dB"""
    # dB = 10 * log10(linear)
    vv = image.select('VV')
    vh = image.select('VH')
    
    vv_db = ee.Image(10).multiply(vv.log10()).rename('VV')
    vh_db = ee.Image(10).multiply(vh.log10()).rename('VH')
    
    # Calcular ratio VV/VH (útil para discriminación de cultivos)
    ratio = vv.divide(vh).log10().multiply(10).rename('VV_VH_ratio')
    
    return image.addBands(vv_db).addBands(vh_db).addBands(ratio) \
        .copyProperties(image, ['system:time_start', 'orbitProperties_pass'])

# Aplicar conversión a dB
s1_db = s1_filtered.map(to_dB)

print("Conversión a dB completada")
print(f"Bandas disponibles: {s1_db.first().bandNames().getInfo()}")

## 6. Filtrado de Speckle

El speckle es un ruido multiplicativo inherente a las imágenes SAR. Se aplicará un filtro espacial para reducirlo [3].

In [ ]:
def apply_speckle_filter(image):
    """Aplica filtro de speckle usando kernel focal"""
    # Filtro de Lee refinado - kernel 7x7
    # Este filtro preserva bordes mientras reduce speckle
    
    vv = image.select('VV')
    vh = image.select('VH')
    ratio = image.select('VV_VH_ratio')
    
    # Aplicar filtro focal medio con kernel cuadrado
    kernel = ee.Kernel.square(radius=3, units='pixels')
    
    vv_filtered = vv.focal_median(kernel=kernel).rename('VV_filtered')
    vh_filtered = vh.focal_median(kernel=kernel).rename('VH_filtered')
    ratio_filtered = ratio.focal_median(kernel=kernel).rename('ratio_filtered')
    
    return image.addBands(vv_filtered).addBands(vh_filtered).addBands(ratio_filtered)

# Aplicar filtro de speckle
s1_filtered_speckle = s1_db.map(apply_speckle_filter)

print("Filtro de speckle aplicado")
print(f"Bandas actualizadas: {s1_filtered_speckle.first().bandNames().getInfo()}")

## 7. Composiciones Temporales

Crear composiciones mensuales para reducir el volumen de datos y mejorar la relación señal-ruido [4].

In [ ]:
def create_monthly_composites(collection, start_date, end_date):
    """Crea composiciones mensuales usando la mediana"""
    # Convertir fechas
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    
    # Calcular número de meses
    n_months = end.difference(start, 'month').round()
    
    def create_composite(month_offset):
        month_start = start.advance(month_offset, 'month')
        month_end = month_start.advance(1, 'month')
        
        monthly = collection.filterDate(month_start, month_end)
        
        composite = monthly.median().set({
            'system:time_start': month_start.millis(),
            'month': month_start.format('YYYY-MM'),
            'n_images': monthly.size()
        })
        
        return composite
    
    # Crear secuencia de meses
    months = ee.List.sequence(0, n_months.subtract(1))
    
    # Mapear sobre los meses
    composites = ee.ImageCollection.fromImages(
        months.map(create_composite)
    )
    
    return composites

# Crear composiciones mensuales
s1_monthly = create_monthly_composites(
    s1_filtered_speckle,
    parametros['fecha_inicio'],
    parametros['fecha_fin']
)

n_composites = s1_monthly.size().getInfo()
print(f"Composiciones mensuales creadas: {n_composites}")

# Mostrar información de las composiciones
first_composite = ee.Image(s1_monthly.first())
print(f"\nPrimera composición:")
print(f"  Mes: {first_composite.get('month').getInfo()}")
print(f"  Número de imágenes: {first_composite.get('n_images').getInfo()}")

## 8. Visualización de Imágenes

In [ ]:
# Crear mapa para visualización
Map = geemap.Map(
    center=[parametros['centroide_lat'], parametros['centroide_lon']], 
    zoom=9
)

# Agregar límites de municipios
Map.add_gdf(municipios_seleccionados, layer_name="Municipios")

# Parámetros de visualización para SAR
vis_params_vv = {
    'min': -25,
    'max': 0,
    'palette': ['blue', 'white', 'green']
}

vis_params_vh = {
    'min': -30,
    'max': -5,
    'palette': ['purple', 'white', 'yellow']
}

# Agregar composición más reciente
last_composite = ee.Image(s1_monthly.sort('system:time_start', False).first())

Map.addLayer(
    last_composite.select('VV_filtered').clip(aoi),
    vis_params_vv,
    'VV - Última composición',
    opacity=0.8
)

Map.addLayer(
    last_composite.select('VH_filtered').clip(aoi),
    vis_params_vh,
    'VH - Última composición',
    opacity=0.8,
    shown=False
)

# Composición RGB falso color (VV, VH, VV/VH)
rgb_composite = last_composite.select(['VV_filtered', 'VH_filtered', 'ratio_filtered'])
Map.addLayer(
    rgb_composite.clip(aoi),
    {'min': [-25, -30, -5], 'max': [0, -5, 5]},
    'RGB Falso Color',
    opacity=0.8,
    shown=False
)

Map

## 9. Análisis de Estadísticas por Municipio

In [ ]:
# Extraer estadísticas de backscatter por municipio
def extract_stats_by_municipality(image, feature):
    """Extrae estadísticas de una imagen para un municipio"""
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ),
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e9
    )
    
    return ee.Feature(None, {
        'municipio': feature.get('municipio'),
        'departamento': feature.get('departamento'),
        'VV_mean': stats.get('VV_filtered_mean'),
        'VV_std': stats.get('VV_filtered_stdDev'),
        'VH_mean': stats.get('VH_filtered_mean'),
        'VH_std': stats.get('VH_filtered_stdDev'),
        'month': image.get('month')
    })

# Convertir municipios a FeatureCollection
def gdf_to_ee_fc(gdf):
    features = []
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom.geom_type == 'Polygon':
            coords = [list(geom.exterior.coords)]
            ee_geom = ee.Geometry.Polygon(coords)
        else:
            coords = [list(poly.exterior.coords) for poly in geom.geoms]
            ee_geom = ee.Geometry.MultiPolygon(coords)
        
        features.append(ee.Feature(ee_geom, {
            'municipio': row['mpio_cnmbr'],
            'departamento': row['dpto_cnmbr']
        }))
    return ee.FeatureCollection(features)

municipios_fc = gdf_to_ee_fc(municipios_seleccionados)

print("Extrayendo estadísticas por municipio...")
print("(Este proceso puede tomar varios minutos)")

# Extraer para la última composición como ejemplo
stats_last_month = municipios_fc.map(
    lambda feat: extract_stats_by_municipality(last_composite, feat)
)

# Convertir a DataFrame
stats_data = stats_last_month.getInfo()['features']
stats_df = pd.DataFrame([f['properties'] for f in stats_data])

print("\nEstadísticas de backscatter por municipio (última composición):")
print(stats_df.to_string(index=False))

## 10. Guardar Colecciones Procesadas

In [ ]:
# Guardar información de las colecciones procesadas
procesamiento_info = {
    'orbit_type': orbit_type,
    'n_images_total': s1_filtered.size().getInfo(),
    'n_composites': n_composites,
    'fechas_disponibles': fechas_disponibles,
    'speckle_filter': 'focal_median_7x7',
    'temporal_aggregation': 'monthly_median',
    'bands': ['VV_filtered', 'VH_filtered', 'ratio_filtered']
}

with open('data/procesamiento_info.json', 'w') as f:
    json.dump(procesamiento_info, f, indent=2)

print("Información de procesamiento guardada")

# Guardar estadísticas
stats_df.to_csv('data/stats_municipios_ultima_composicion.csv', index=False)
print("Estadísticas guardadas en: data/stats_municipios_ultima_composicion.csv")

## Resumen

En este notebook se completaron las siguientes tareas:

1. ✓ Acceso a la colección Sentinel-1 GRD en Google Earth Engine
2. ✓ Filtrado por modo de adquisición (IW), polarización (VV+VH) y órbita
3. ✓ Análisis de disponibilidad temporal de datos
4. ✓ Conversión de backscatter a escala logarítmica (dB)
5. ✓ Cálculo de ratio VV/VH
6. ✓ Aplicación de filtro de speckle (focal median)
7. ✓ Creación de composiciones mensuales
8. ✓ Visualización de imágenes procesadas
9. ✓ Extracción de estadísticas por municipio
10. ✓ Exportación de datos procesados

### Colecciones disponibles para análisis de cambios:
- `s1_filtered_speckle`: Colección completa con filtro de speckle
- `s1_monthly`: Composiciones mensuales (recomendado para detección de cambios)

**Próximo paso**: Notebook 3 - Análisis de detección de cambios usando algoritmos estadísticos